# Neuromorphic computing assignment 3 option B

### By S.W. Keemink, 2023-09-21

In the previous lecture you saw that if you know the neuronal activation functions (and they are rich enough), you can make a spiking neural network do a range of input-output computations of the form $y = f(x)$, where $x$ is the network intput and $y$ is the network output. The underlying theory is the neural engineering framework (or NEF) --- used in the NENGO software.

**Delivarable:**  The deliverable is a pdf export of this notebook with the requested plots and questions answered.


### Basic Imports

In [ ]:
import numpy as np
import numba as nb # highly recommend installing this, it will make things much faster
import matplotlib.pyplot as plt

%matplotlib inline

### System setup
We will consider a network of $N$ integrate and fire neurons the voltage of neuron $i$ is given by

$$\frac{dv_i}{dt}(t) = -v_i + f_i x(t) + b_i,$$

where $v$ is the neuron's voltage, $f_i$ are is its input weight, $x(t)$ is the input into the neuron, and $b_i$ its bias. When a threshold $T_i$ is reached, a spike is emitted, and the neuron's voltage is reset to 0.

A network of such neurons can be described in vector format as:
$$\frac{d\mathbf V}{dt}(t) = -\mathbf V + \mathbf F x(t) + \mathbf b,$$

where $\mathbf V = [v_0, ..., v_N]^\top$, $\mathbf F = [f_0, ..., f_N]^\top$, and $\mathbf b = [b_0, ..., b_N]^\top$. Note that one would normally also add some kind of membrane-leak time-constant, but we will leave that out to keep things a bit simpler.

We will filter the emitted spiketrains by an exponential filter as follows:
$$\frac{dr_i}{dt}(t) = -r_i(t) + s(t),$$
where $r_i$ are the filtered spike-trains, and $s(t)$ are the spiketrains themselves.

I will provide a simple Euler simulator for a network of such neurons.

In [ ]:
@nb.jit(nopython=True)
def run_LIF_network(x, F, b, T, dt=1e-3):
    """Simulates a network of leaky integrate-and-fire neurons (as described above).
    
    Parameters
    ----------
    x : array (shape=nT)
        Input signal. nT=number of timepoints.
    F : array (shape=N)
        Input weights. N=number of neurons.
    b : array (shape=N)
        Biases
    T : array (shape=N)
        Thresholds
    dt : float
        Simulation timestep
        
    Output
    ------
    array
        Array of shape (N, nT) containing the simulated voltages
    array
        Array of shape (N, nT) containing the spikes
    array
        Array of shape (N, nT) containing the time-filtered spikes
        
    """
    # recover number of timepoints
    nT = len(x)
    N = len(F)
    
    # pre-define states
    V = np.zeros((N, nT))
    s = np.zeros((N, nT))
    r = np.zeros((N, nT))
    
    # simulate with Euler
    for i in range(1, nT):
        # update voltage
        V[:, i] = V[:, i-1] + dt*(-V[:, i-1] + F*x[i-1] + b)
        
        # update filtered spike trains
        r[:, i] = r[:, i-1] + dt*(-r[:, i-1]) + s[:, i-1]
        
        # detect spikes and reset
        spiking_neurons = np.arange(N)[V[:, i] >= T]
        V[spiking_neurons, i] = 0 # reset
        s[spiking_neurons, i] = 1 # set spikes
        
    return V, s, r

Simulation of a single neuron example

In [ ]:
# Time settings
dt = 1e-3 # simulation timestep in ms
Tend = 10 # simulation end in ms
times = np.arange(0, Tend, dt) # time vector stored for plotting
nT = len(times) # number of timepoints

# Network parameters
N = 10 # number of neurons
F = np.ones(N)*0.4 # input weights
b = np.zeros(N)*2 # biases
T = np.ones(N) # thresholds

# Define input
x = np.ones(nT)*5 # 1D input signal (you can define KD signals by changing the shape to (K, nT))

# Simulate
V, s, r = run_LIF_network(x, F, b, T, dt=dt)

# Plot a few things (only works for the first neuron at the moment)
plt.subplot(311)
plt.plot(times, r[0, :])
plt.ylabel('Filtered spikes')
plt.subplot(312)
plt.plot(times, s[0, :])
plt.ylabel('Spikes')
plt.subplot(313)
plt.plot(times, V[0, :])
plt.axhline(T[0], color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.legend(['Voltage', 'Threshold'])

Note that the filtered spike train needs a moment to stabilize at a fixed value (which is the correct average value)

### Exercise 1: Explore input-output functions of spiking neurons


1. By running several separate trials with different inputs $x(t)$ between -5 and 5 (held fixed at a given level in each trial), show what the average input output-function is. [0.5 points]

For this you can measure what the average output $r(t)$ is within the range where it is stabilized (see above).

The analytical solution for the firing rate for the above neural model is:

$$r = [ (\ln(\frac{f_i x + b_i}{f_i + b_i - T_i}))^{-1} ]^{+} $$

Which we implement below:

In [1]:
def LIF_io(x, f, b, T):
    out = np.log((f*x+b) / (f*x+b-T))
    rate = 1/out
    rate[out<=0] = 0
    rate[np.isnan(rate)] = 0
    return rate

2. Confirm that your estimated input-output function from 1 is the same as this theoretical one. [0.5 points]

3. Simulate a network of 10 neurons with random biases and input weights, and by plotting the estimated input-output curves with the theoretical ones, confirm that this works for a range of values. [1 points]

### Exercise 2: Decoding the input from a good basis
In this exercise we will adjust the input weights and biases to create a good basis set of activation functions. We will then use these to decode the input that was provided from the neural activities.


1. Generate 50 neurons with basis functions which span a range of x from -5 to 5 well. The functions could be randomly sampled, or evenly spaced, or as you wish. If later exercises do not work well, it might be that you did not get a good enough basis in this step, and you could also consider using a larger set of neurons. [1 point]

Consider a specific read-out of the filtered spiketrains given by $y = \mathbf D \mathbf r$. Here $\mathbf D$ are some decoding weights, and $\mathbf r = [r_1, ..., r_N]^\top$ are the filtered spiketrains. In the NEF the optimal decoder is found which minimizes the following loss across a large set of possible inputs $x$:
$$L = ||\mathbf X - \mathbf D \mathbf R||^2_2 + \lambda||\mathbf D||^2_2,$$

where $\mathbf X = [x_1, ..., x_\text{nSamples}]^\top$ is a representative set of input samples, $\mathbf R = [r_1, ..., r_\text{nSamples}]^\top$ are the resulting activities. The first term represents the error between the decoded activities and the input, and the second part is a regularizer which makes sure the decoding weights can't get arbitrarily large (and keeps the problem constrained enough to be solvable). Normally with spiking networks this would be quite expensive to estimate (as you would have to simulate all your neurons for each input), but because we have an analytical expression for the output rates this simplifies things.

We  now want to find the decoding weights $\mathbf D$ which optimizes our loss --- very machine-learnery. And indeed one way to that is to calculate the gradient of the loss with respect to $\mathbf D$ and perform gradient descent. However, again luckily, there exists a linear regression solution to this loss, given by:
$$\mathbf D = \mathbf {XR}^\top (\mathbf{RR}^\top + \lambda \mathbf I)^{-1},$$
where $\mathbf I$ is the identity matrix. $\lambda$ should be set to some small value.

2. Using your set of weights defined in 1., calculate the decoder as above. Use a range of input samples between -5 and 5. Demonstrate that $\mathbf D$ was calculated correctly by comparing $\mathbf X$ with $\mathbf D \mathbf R$. [2 points]

Note: a matrix inverse can be calculated with np.linalg.inv(X). Matrix multiplications can be done by X@Y or np.dot(X, Y).

Note 2: $\mathbf X$ should be of shape (nSamples), and $\mathbf R$ should be of shape (N, nSamples), where N is the number of neurons.


3. Demonstrate that you can use the decoder on a network of spiking neurons. Do this by stepping the input $x(t)$ from -5 to 5 with steps of 1, and keeping the signal at each step for at least 10 ms. [2 points]

Note that we need to keep the stimulus still enough for the read-out and spiking dynamics to catch up properly. It is out of scope for this exercise to make this work better, but it is not a big problem in practice to fix this. For example, one could use much faster read-out and voltage dynamics --- but this then also comes at the cost of needing many, many more spikes.

### Exercise 3: Doing nonlinear computations
Doing now a nonlinear transformation ($x = f(x)$) is next fairly simple, and follows the same principle. We now optimize $$L = ||f(\mathbf X) - \mathbf D \mathbf R||^2_2 + \lambda||\mathbf D||^2_2$$ instead (note the functional added in), leading to a decoder given by $$\mathbf D^f =  {f(\mathbf X)R}^\top (\mathbf{RR}^\top + \lambda \mathbf I)^{-1}.$$

1. Choose at least 2 different nonlinear functions f(), and find their decoders. Demonstrate that they work by comparing $\mathbf X$ with $\mathbf D^f \mathbf R$ and with $f(\mathbf X)$. [2 points]


2. Show that this still works in the spiking networks. [1 points]

# Bonus assignment (no points, just for interest)
Above we considered everything for a 1 dimensional signal $x(t)$. How would you extent everything to consider a 2D or higher input signal? What would this mean for the input-output functions and the decoder?